In [410]:
import pandas as pd; import numpy as np; from shapely import geometry; import geopandas as gpd
import json; from pprint import pprint; import sys
%matplotlib inline

# Breaking everything down to Buildings

In [411]:
buildings = pd.read_csv('../raw_data/ZayoHackathonData_Buildings.csv')
sites     = pd.read_csv('../raw_data/ZayoHackathonData_Sites.csv')
accounts  = pd.read_csv('../raw_data/ZayoHackathonData_Accounts.csv')
opportunities = pd.read_csv('../raw_data/ZayoHackathonData_Opportunities.csv')
products      = pd.read_csv('../raw_data/ZayoHackathonData_Services.csv')

In [412]:
print(buildings.columns)
buildings.head(1)

Index(['Building ID', 'Market', 'Street Address', 'City', 'State',
       'Postal Code', 'Latitude', 'Longitude', 'On Zayo Network Status',
       'Net Classification', 'Type', 'Network Proximity',
       ' Estimated Build Cost '],
      dtype='object')


,Building ID,Market,Street Address,City,State,Postal Code,Latitude,Longitude,On Zayo Network Status,Net Classification,Type,Network Proximity,Estimated Build Cost
0,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,33.728521,-84.418751,Not on Zayo Network,Fiber Only,Office - Multi Tenant,116.91,"$13,507.30"


In [413]:
buildings['est_cost'] = buildings[' Estimated Build Cost '].replace('[\$,)]','', regex=True ).astype(float)

In [414]:
buildings.head(3)

,Building ID,Market,Street Address,City,State,Postal Code,Latitude,Longitude,On Zayo Network Status,Net Classification,Type,Network Proximity,Estimated Build Cost,est_cost
0,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,33.728521,-84.418751,Not on Zayo Network,Fiber Only,Office - Multi Tenant,116.91,"$13,507.30",13507.30
1,Bldg-115473,Atlanta,555 Old Norcross Rd,Lawrenceville,GA,30046,33.946700,-83.995000,Not on Zayo Network,Fiber Only,Office - Multi Tenant,28323.03,"$689,752.72",689752.72
2,Bldg-115475,Atlanta,3159 Royal Dr,Alpharetta,GA,30005,34.056608,-84.265246,On Zayo Network,Fiber Only,Office - Multi Tenant,979.44,"$33,506.56",33506.56


In [415]:
sites.head(1)

,Site ID,Account ID,Building ID,Market,Site Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Site-000001,Acct-000025,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,116.91,Not on Zayo Network


In [416]:
accounts.head(1)

,Account ID,Industry,Vertical,Total BRR,AnnualRevenue,NumberOfEmployees,DandB Revenue,DandB Total Employees
0,Acct-000273,Telecommunications,Wireless,"$6,084,906.00","$248,747,365.62",397,"$356,943,618.45",397


In [417]:
opportunities.head(1)

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network


In [418]:
df = accounts.merge(sites,on="Account ID")

In [419]:
df2 = df.merge(buildings, on="Building ID")

In [420]:
df2.head(1)

,Account ID,Industry,Vertical,Total BRR,AnnualRevenue,NumberOfEmployees,DandB Revenue,DandB Total Employees,Site ID,Building ID,...,State_y,Postal Code_y,Latitude,Longitude,On Zayo Network Status_y,Net Classification,Type,Network Proximity_y,Estimated Build Cost,est_cost
0,Acct-000273,Telecommunications,Wireless,"$6,084,906.00","$248,747,365.62",397,"$356,943,618.45",397,Site-000061,Bldg-115571,...,GA,30309,33.800922,-84.393502,On Zayo Network,Fiber Only,Office - Multi Tenant,198.92,"$15,967.60",15967.6


In [421]:
df2.head(2).columns

Index(['Account ID', 'Industry', 'Vertical', ' Total BRR ', ' AnnualRevenue ',
       'NumberOfEmployees', ' DandB Revenue ', 'DandB Total Employees',
       'Site ID', 'Building ID', 'Market_x', 'Site Address', 'City_x',
       'State_x', 'Postal Code_x', 'Network Proximity_x',
       'On Zayo Network Status_x', 'Market_y', 'Street Address', 'City_y',
       'State_y', 'Postal Code_y', 'Latitude', 'Longitude',
       'On Zayo Network Status_y', 'Net Classification', 'Type',
       'Network Proximity_y', ' Estimated Build Cost ', 'est_cost'],
      dtype='object')

In [422]:
opportunities.head()

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network
1,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,Bldg-016855,Atlanta,300 Satellite Blvd NW,Suwanee,GA,30024,374.79,On Zayo Network
2,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,Bldg-109003,Atlanta,305 Satellite Blvd NW,Suwanee,GA,30024,250.74,On Zayo Network
3,Opp-000003,Acct-000011,5 - Accepted,True,True,7/1/16,36.0,NaN,New Service,Dark Fiber - Metro,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network
4,Opp-000003,Acct-000011,5 - Accepted,True,True,7/1/16,36.0,NaN,New Service,Dark Fiber - Metro,Bldg-038069,Atlanta,1100 Abernathy Rd NE,Atlanta,GA,30328,151.21,On Zayo Network


In [423]:
x = df2.groupby("Building ID").agg({
        "Latitude"  : {'lat': lambda x: x.values[0]},
        "Longitude" : {'lon': lambda x: x.values[0]},
        "Account ID": {"Number Accounts": pd.Series.nunique},
        "est_cost"  : {"BuildingCost" : sum},
        "Site ID" : {"Number Sites" : pd.Series.nunique},
        "On Zayo Network Status_y": {
            "netStat" : lambda x: x.values[0]=='On Zayo Network'}
    })
x.columns = x.columns.get_level_values(1)

In [424]:
opp_count = pd.DataFrame(opportunities.query('StageName!="Closed - Lost"')['Building ID'].value_counts())

In [425]:
opp_count.head()

,Building ID
Bldg-014866,417
Bldg-012582,175
Bldg-017035,148
Bldg-065056,94
Bldg-127057,76


In [426]:
gdf = gpd.GeoDataFrame(x)
gdf.geometry = gdf.apply(lambda row: geometry.Point(row.lon, row.lat), axis=1)

In [427]:
print(len(gdf))
gdf.head(2)

17209


,lat,lon,BuildingCost,Number Sites,netStat,Number Accounts,geometry
Building ID,,,,,,,
Bldg-011543,39.598945,-104.860338,71994.9,3,True,3,POINT (-104.860338 39.598945)
Bldg-012582,33.755431,-84.391561,3430141.0,286,True,275,POINT (-84.391561 33.755431)


## Add opportunities (lookup)

In [428]:
gdf['opportunity'] = 0

In [429]:
opportunities.columns

Index(['Opportunity ID', 'Account ID', 'StageName', 'IsClosed', 'IsWon',
       'CreatedDate', 'Term in Months', 'Service', 'Opportunity Type',
       'Product Group', 'Building ID', 'Market', 'Street Address', 'City',
       'State', 'Postal Code', 'Network Proximity', 'On Zayo Network Status'],
      dtype='object')

In [430]:
gdf.head(2)

,lat,lon,BuildingCost,Number Sites,netStat,Number Accounts,geometry,opportunity
Building ID,,,,,,,,
Bldg-011543,39.598945,-104.860338,71994.9,3,True,3,POINT (-104.860338 39.598945),0
Bldg-012582,33.755431,-84.391561,3430141.0,286,True,275,POINT (-84.391561 33.755431),0


In [431]:
opportunities.head(2)

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network
1,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,Bldg-016855,Atlanta,300 Satellite Blvd NW,Suwanee,GA,30024,374.79,On Zayo Network


In [432]:
opportunities['b_id'] = opportunities['Building ID']
products['b_id'] = products['Building ID']

In [433]:
print(opportunities.columns)
opportunities.head(2)

Index(['Opportunity ID', 'Account ID', 'StageName', 'IsClosed', 'IsWon',
       'CreatedDate', 'Term in Months', 'Service', 'Opportunity Type',
       'Product Group', 'Building ID', 'Market', 'Street Address', 'City',
       'State', 'Postal Code', 'Network Proximity', 'On Zayo Network Status',
       'b_id'],
      dtype='object')


,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,Building ID,Market,Street Address,City,State,Postal Code,Network Proximity,On Zayo Network Status,b_id
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,Bldg-012582,Atlanta,56 Marietta St NW,Atlanta,GA,30303,66.45,On Zayo Network,Bldg-012582
1,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,Bldg-016855,Atlanta,300 Satellite Blvd NW,Suwanee,GA,30024,374.79,On Zayo Network,Bldg-016855


In [393]:
def buildFeatureWithOpportunity(row):
    
    opps = opportunities.query("b_id=='{0}'".format(row.name))    
    op = [0,0,0,0,0,0,0]

    if len(opps):
        op[0] = 1
        val_counts = opps.StageName.value_counts()
        for idx, stage in enumerate(['1 - Working','2 - Best Case','3 - Committed','4 - Closed', '5 - Accepted', 'Closed - Lost']):
            if stage in val_counts:
                op[idx+1] = int(val_counts[stage])
    
    feat = {"type":"Feature",
            "properties":{
                    "opp_0":    op[0],
                    "opp_1":    op[1],
                    "opp_2":    op[2],
                    "opp_3":    op[3],
                    "opp_4":    op[4],
                    "opp_5":    op[5],
                    "opp_lost": op[6]
                }
            }
    return feat

In [435]:
products.columns

Index(['Service ID', 'Account ID', ' Total MRR ', ' Netx MRC ',
       'Product Group', 'Status', 'Building ID', 'Street Address', 'City',
       'State', 'Postal Code', 'Country', 'b_id'],
      dtype='object')

In [436]:
products['total_mrr'] = products[' Total MRR '].replace('[\$,()]','', regex=True )
products.total_mrr = products.total_mrr.replace('[-]',0, regex=True ).astype(float)

products['netx_mrc'] = products[' Netx MRC '].replace('[\$,()]','', regex=True )
products.netx_mrc = products.netx_mrc.replace('[-]',0, regex=True ).astype(float)

In [454]:
buildings

,Building ID,Market,Street Address,City,State,Postal Code,Latitude,Longitude,On Zayo Network Status,Net Classification,Type,Network Proximity,Estimated Build Cost,est_cost
0,Bldg-115472,Atlanta,953 Donnelly Ave SW,Atlanta,GA,30310,33.728521,-84.418751,Not on Zayo Network,Fiber Only,Office - Multi Tenant,116.91,"$13,507.30",13507.30
1,Bldg-115473,Atlanta,555 Old Norcross Rd,Lawrenceville,GA,30046,33.946700,-83.995000,Not on Zayo Network,Fiber Only,Office - Multi Tenant,28323.03,"$689,752.72",689752.72
2,Bldg-115475,Atlanta,3159 Royal Dr,Alpharetta,GA,30005,34.056608,-84.265246,On Zayo Network,Fiber Only,Office - Multi Tenant,979.44,"$33,506.56",33506.56
3,Bldg-115476,Atlanta,1593 Northeast Expy NE,Atlanta,GA,30329,33.831700,-84.323900,Not on Zayo Network,Type 2,Neutral Data Center,177.41,"$15,322.30",15322.30
4,Bldg-115501,Atlanta,Glenridge Dr NE & Hammond Dr NE,Sandy Springs,GA,30328,33.918987,-84.364091,Not on Zayo Network,Stubbed,Splice Point,22.89,"$10,686.70",10686.70
5,Bldg-115514,Atlanta,2500 Cumberland Pkwy SE,Atlanta,GA,30339,33.867537,-84.475962,Not on Zayo Network,Fiber Only,Office - Multi Tenant,2620.30,"$72,887.20",72887.20
6,Bldg-115480,Atlanta,Northside Dr & Sagamore Dr,Atlanta,GA,30305,33.821757,-84.405760,On Zayo Network,Fiber Only,Splice Point,19.09,"$10,572.70",10572.70
7,Bldg-115549,Atlanta,100 Peachtree St NE,Atlanta,GA,30303,33.756924,-84.388650,On Zayo Network,Fiber Only,Office - Multi Tenant,519.98,"$22,479.52",22479.52
8,Bldg-115550,Atlanta,1000 Holcomb Woods Pkwy - Bldg 444,Roswell,GA,30076,34.022335,-84.309742,On Zayo Network,Fiber Only,Office - Multi Tenant,562.01,"$23,488.24",23488.24
9,Bldg-115551,Atlanta,105 Collier Rd NW,Atlanta,GA,30309,33.809081,-84.396315,On Zayo Network,Fiber Only,Hospital,356.89,"$20,706.70",20706.70


In [452]:
products.to_csv('products.csv')

In [453]:
buildings_and_products = products.merge(buildings, on="b_id", how="outer")

KeyError: 'b_id'

In [451]:
len(buildings_and_products)

125049

In [406]:
# Adding building products...
product_list = list(products['Product Group'].unique())
#over_ride
product_list = ['Dark Fiber - Metro',
    'Ethernet',
    'IP Services',
    'SONET',
    'FTT - Dark Fiber',
    'ISP',
    'Managed WAN-LAN',
    'FTT - Ethernet',
    'Dark Fiber - Long Haul',
    'FTT - Small Cell',
    'Wavelengths - Long Haul',
    'Wavelengths - Metro',
    'zColo']

def buildProducts(row):
    
    #prods is a dataframe of products for this building
    prods = products.query("b_id=='{0}'".format(row.name))
    
    if len(prods)==0:
        b_rev = 0
        val_counts={}
    else:
        #total building revenue potential -- not at a per product level
        b_rev = int(prods.total_mrr.sum() - prods.netx_mrc.sum())
        val_counts = prods['Product Group'].value_counts()
    
    #Build an empty hash for this buildings products:
    building_products = {}

    for p_name in product_list:
        if p_name in val_counts:
            building_products[p_name] = int(val_counts[p_name])
        else:
            building_products[p_name] = 0
    
    return b_rev, building_products

In [407]:
def make_geojson(row):
    geom = {"type":"Point","coordinates":[]}
    if (not np.isnan(row.lon) and not np.isnan(row.lat)):
        geom['coordinates'].append(row.lon)
        geom['coordinates'].append(row.lat)
    else:
        return false
    
    if row['netStat']:
        cost = 0
        netStat = 1
    else:
        cost = row['BuildingCost']
        netStat = 0
    
    feat = buildFeatureWithOpportunity(row)
    feat["geometry"]  = geom
    
    feat["properties"]
    feat["properties"]["sites"]       = row["Number Sites"],
    feat["properties"]["accounts"]    = row["Number Accounts"],
    feat["properties"]["cost"]        = cost
    feat["properties"]["netStat"]     = netStat
    
    b_rev, prods = buildProducts(row)
    
    feat["properties"]["b_rev"] = b_rev
    
    feat["properties"] = {**feat["properties"], **prods}

    return feat

In [408]:
#Create GeoJSON
features = []
total = len(gdf[gdf.netStat])
cnt = 0;
for idx,row in gdf[gdf.netStat].iterrows():
    r = make_geojson(row)
    if r:
        features.append(r)
    sys.stderr.write("\r{0} of {1}".format(cnt, total))
    cnt += 1

with open('../docs/data/onNetwork_buildings_sm.geojson','w') as oFile:
    json.dump({"type":"FeatureCollection","features":features},oFile,indent=4)

3413 of 3414

In [409]:
#Create GeoJSON
features = []
total = len(gdf[~gdf.netStat])
cnt = 0;
for idx,row in gdf[~gdf.netStat].iterrows():
    r = make_geojson(row)
    if r:
        features.append(r)
    sys.stderr.write("\r{0} of {1}".format(cnt, total))
    cnt += 1

with open('../docs/data/offNetwork_buildings_sm.geojson','w') as oFile:
    json.dump({"type":"FeatureCollection","features":features},oFile,indent=4)

13794 of 13795